In [ ]:
import os
import pandas as pd

playersShiftDirectory = "../MLBDATA/Raw/ShiftData"
seasonsFilePath       = "../MLBDATA/Raw/BatterData/BattingData2015-2024NoCovid.csv"
archetypesFilePath    = "../MLBDATA/Processed/BatterData/PlayerArchetypesClusters2016-2022NoCovid.csv"
firstYear             = 2015
lastPreBanYear        = 2022
firstPostBanYear      = 2023
outputFile            = "../MLBDATA/Processed/BatterData/ModelingBattingData.csv"

dfSeasons     = pd.read_csv(seasonsFilePath)
dfArchetypes  = pd.read_csv(archetypesFilePath)

dfPlayerShift = pd.read_csv(os.path.join(playersShiftDirectory, "POSPlayer2016.csv"))
dfPlayerShift.drop(columns=["team_id", "total_pa"], inplace=True)
dfPlayerShift.rename(columns={"name": "last_name, first_name"}, inplace=True)

for fn in os.listdir(playersShiftDirectory):
    if fn.startswith("POSPlayer") and fn.endswith(".csv") and fn != "POSPlayer2016.csv":
        tmp = pd.read_csv(os.path.join(playersShiftDirectory, fn))
        tmp.drop(columns=["team_id", "total_pa"], inplace=True)
        tmp.rename(columns={"name": "last_name, first_name"}, inplace=True)
        dfPlayerShift = pd.concat([dfPlayerShift, tmp], ignore_index=True)

dfPlayerShift = dfPlayerShift[dfPlayerShift["year"] <= lastPreBanYear]

dfPlayerShift.rename(columns={"pa_shift_percent": "shift_rate"}, inplace=True)

df_avg_shift = (
    dfPlayerShift
    .groupby("last_name, first_name", as_index=False)["shift_rate"]
    .mean()
)

extremely_shifted_players = df_avg_shift[df_avg_shift["shift_rate"] > 50]

def normalize_name(s):
    return s.str.replace(r"[,\s]+", "", regex=True).str.lower()

dfArchetypes["name_key"] = normalize_name(dfArchetypes["last_name, first_name"])
extremely_shifted_players["name_key"] = normalize_name(extremely_shifted_players["last_name, first_name"])

merged_shifted = pd.merge(
    extremely_shifted_players,
    dfArchetypes[["name_key", "cluster"]],
    on="name_key",
    how="inner"
)

extreme_counts_by_cluster = (
    merged_shifted
    .groupby("cluster")["name_key"]
    .nunique()
    .reset_index(name="num_extremely_shifted")
)

print(extreme_counts_by_cluster)

   cluster  num_extremely_shifted
0        0                      2
1        1                     36
2        2                     61


C:\Users\Andrew\AppData\Local\Temp\ipykernel_8164\3894287357.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extremely_shifted_players["name_key"] = normalize_name(extremely_shifted_players["last_name, first_name"])
